In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [7]:
train_features = pd.read_csv('/kaggle/input/dacfugo/user_features_train.csv')
target = pd.read_csv('/kaggle/input/dacfugo/targets_train.csv')

In [8]:
merged_df = pd.merge(train_features, target)

retention_columns = [col for col in merged_df.columns if col.startswith('Retention')]
merged_df['TotalRetention'] = merged_df[retention_columns].sum(axis=1)

level_advanced_count_columns = [col for col in merged_df.columns if col.startswith('LevelAdvancedCount')]
merged_df['TotalLevelAdvancedCount'] = merged_df[level_advanced_count_columns].sum(axis=1)

level_duration_columns = [col for col in merged_df.columns if col.startswith('Level_')]
merged_df['TotalLevelDuration'] = merged_df[level_duration_columns].fillna(0).sum(axis=1)

ad_revenue_columns = [col for col in merged_df.columns if col.startswith('AdRevenue')]
merged_df['TotalAdRevenue'] = merged_df[ad_revenue_columns].sum(axis=1)

In [9]:
merged_df.head()

,ID,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,RetentionD8,RetentionD9,RetentionD10,RetentionD11,RetentionD12,RetentionD13,RetentionD14,RetentionD15,LevelAdvancedCountD0,LevelAdvancedCountD1,LevelAdvancedCountD2,LevelAdvancedCountD3,LevelAdvancedCountD4,LevelAdvancedCountD5,LevelAdvancedCountD6,LevelAdvancedCountD7,LevelAdvancedCountD8,LevelAdvancedCountD9,LevelAdvancedCountD10,LevelAdvancedCountD11,LevelAdvancedCountD12,LevelAdvancedCountD13,LevelAdvancedCountD14,LevelAdvancedCountD15,Level_1_Duration,Level_2_Duration,Level_3_Duration,Level_4_Duration,Level_5_Duration,Level_6_Duration,Level_7_Duration,Level_8_Duration,Level_9_Duration,Level_10_Duration,AdRevenueD0,AdRevenueD1,AdRevenueD2,AdRevenueD3,AdRevenueD4,AdRevenueD5,AdRevenueD6,AdRevenueD7,AdRevenueD8,AdRevenueD9,AdRevenueD10,AdRevenueD11,AdRevenueD12,AdRevenueD13,AdRevenueD14,AdRevenueD15,IAPRevenueD0,IAPRevenueD1,IAPRevenueD2,IAPRevenueD3,IAPRevenueD4,IAPRevenueD5,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,TotalRetention,TotalLevelAdvancedCount,TotalLevelDuration,TotalAdRevenue
0,0,3.314099,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0,69.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2,3,131.0,0.000000
1,1,1.681524,True,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,5,0,0,0,0,0,3,5,0,2,0,0,0,0,0,0,13.0,91.0,39.0,79.0,180.0,89.0,124.0,118.0,35.0,117.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.008674,0.0,0.010218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018892,4,15,885.0,0.018892
2,2,10.718750,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,63.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1,3,157.0,0.000000
3,3,5.100000,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,19,8,9,0,0,0,0,0,0,0,0,0,0,0,0,0,23.0,141.0,131.0,118.0,77.0,107.0,77.0,182.0,42.0,156.0,0.00215,0.019159,0.025341,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046650,3,36,1054.0,0.046650
4,4,2.091409,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,52.0,30.0,84.0,139.0,96.0,268.0,97.0,44.0,122.0,0.01468,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014680,1,15,940.0,0.014680


In [10]:
%%time
import altair as alt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error

features = ['TotalRetention', 'TotalLevelAdvancedCount', 'TotalLevelDuration', 'TotalAdRevenue']
target = 'TARGET'

x_train= merged_df[features]
y_train = merged_df[target]



model = RandomForestRegressor(random_state=42)



param_grid = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 3],  
    'max_features': ['sqrt']
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(x_train, y_train)


print("En iyi hiperparametreler: ", grid_search.best_params_)
print("En iyi negatif MSE: ", grid_search.best_score_)



Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time= 1.2min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 2.5min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 4.6min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time= 1.2min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time= 1.2min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time= 2.3min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time= 2.3min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; 

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time= 1.2min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 2.5min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 4.6min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 4.7min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time= 2.3min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time= 4.4min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time= 1.1min
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time= 2.3min
[CV] END max_depth=10, max_features=sqrt

In [11]:
scores = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = (-scores)**0.5
avg_rmse = rmse_scores.mean()
print(f"Average RMSE: {avg_rmse}")

[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time= 3.7min
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time= 7.2min
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time= 2.5min
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time= 2.5min
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 5.0min
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time= 9.9min
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=10.1min
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time= 5.1min
[CV] END max_depth=30, max_features=sqrt

KeyboardInterrupt: 

In [12]:
import pandas as pd
merged_df1 = pd.read_csv('/kaggle/input/dacfugo/user_features_test.csv')

In [14]:
retention_columns = [col for col in merged_df1.columns if col.startswith('Retention')]
merged_df1['TotalRetention'] = merged_df1[retention_columns].sum(axis=1)

level_advanced_count_columns = [col for col in merged_df1.columns if col.startswith('LevelAdvancedCount')]
merged_df1['TotalLevelAdvancedCount'] = merged_df1[level_advanced_count_columns].sum(axis=1)

level_duration_columns = [col for col in merged_df.columns if col.startswith('Level_')]
merged_df1['TotalLevelDuration'] = merged_df[level_duration_columns].fillna(0).sum(axis=1)

ad_revenue_columns = [col for col in merged_df1.columns if col.startswith('AdRevenue')]
merged_df1['TotalAdRevenue'] = merged_df1[ad_revenue_columns].sum(axis=1)

print(merged_df1.head().to_markdown(index=False, numalign="left", stralign="left"))

| ID     | first_prediction   | RetentionD0   | RetentionD1   | RetentionD2   | RetentionD3   | RetentionD4   | RetentionD5   | RetentionD6   | RetentionD7   | RetentionD8   | RetentionD9   | RetentionD10   | RetentionD11   | RetentionD12   | RetentionD13   | RetentionD14   | RetentionD15   | LevelAdvancedCountD0   | LevelAdvancedCountD1   | LevelAdvancedCountD2   | LevelAdvancedCountD3   | LevelAdvancedCountD4   | LevelAdvancedCountD5   | LevelAdvancedCountD6   | LevelAdvancedCountD7   | LevelAdvancedCountD8   | LevelAdvancedCountD9   | LevelAdvancedCountD10   | LevelAdvancedCountD11   | LevelAdvancedCountD12   | LevelAdvancedCountD13   | LevelAdvancedCountD14   | LevelAdvancedCountD15   | Level_1_Duration   | Level_2_Duration   | Level_3_Duration   | Level_4_Duration   | Level_5_Duration   | Level_6_Duration   | Level_7_Duration   | Level_8_Duration   | Level_9_Duration   | Level_10_Duration   | AdRevenueD0   | AdRevenueD1   | AdRevenueD2   | AdRevenueD3   | AdRevenueD4   | AdRevenue

In [15]:
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(merged_df1[features])

In [16]:
sub = pd.read_csv('/kaggle/input/dacsample/sample_submission.csv')
sub['price'] = y_pred
sub.drop('TARGET', axis=1)

,ID,price
0,878594,0.085698
1,878595,0.013856
2,878596,0.013101
3,878597,0.369130
4,878598,0.116308
...,...,...
585725,1464319,0.311589
585726,1464320,4.292298
585727,1464321,1.754886
585728,1464322,0.031641


In [17]:
sub.drop('TARGET', axis = 1)

,ID,price
0,878594,0.085698
1,878595,0.013856
2,878596,0.013101
3,878597,0.369130
4,878598,0.116308
...,...,...
585725,1464319,0.311589
585726,1464320,4.292298
585727,1464321,1.754886
585728,1464322,0.031641


In [18]:
sub.to_csv("submission.csv", index = False)